In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import random

model = tf.keras.Sequential()

model.add(layers.Dense(7*7*1024, use_bias=False, input_shape=(20,)))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU())

model.add(layers.Reshape((7, 7, 1024)))
assert model.output_shape == (None, 7, 7, 1024)

model.add(layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False))
assert model.output_shape == (None, 14, 14, 512)
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU())

model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
assert model.output_shape == (None, 28, 28, 256)
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU())

model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
assert model.output_shape == (None, 56, 56, 128)
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU())

model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
assert model.output_shape == (None, 112, 112, 64)
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU())

model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
assert model.output_shape == (None, 224, 224, 32)
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU())

model.add(layers.Conv2D(3, (7, 47), padding='valid', use_bias=False, activation='tanh'))
assert model.output_shape == (None, 218,178,3)

model.add(layers.Flatten())
assert model.output_shape == (None, 218*178*3)


model2 = tf.keras.Sequential()

# Reshape the flattened input into a 3D image
model2.add(layers.Reshape((218, 178, 3), input_shape=(218*178*3,)))

model2.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
model2.add(layers.LeakyReLU())
model2.add(layers.Dropout(0.3))

model2.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
model2.add(layers.LeakyReLU())
model2.add(layers.Dropout(0.3))

model2.add(layers.Flatten())
model2.add(layers.Dense(1))

# For model (generator)
generator_optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(optimizer=generator_optimizer, loss='binary_crossentropy')

# For model2 (discriminator)
discriminator_optimizer = tf.keras.optimizers.Adam(0.005)
model2.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')






In [5]:
# Binary Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    # Loss for real images, should be an array of ones
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)

    # Loss for fake images, should be an array of zeros
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)

    # Total loss is the sum of the above losses
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    # The generator tries to get the discriminator to output ones for its generated images
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [6]:
from pathlib import Path
from PIL import Image
import numpy as np

In [ ]:
# Suppose your images are stored in a numpy array
# images = ...
# Load and preprocess images
image_folder = Path('/content/dataset/imgsd')
images = []
for image_path in image_folder.glob('*.*'):
    image = Image.open(image_path)
    image = image.resize((178, 218))
    image = np.array(image)
    images.append(image)
images = np.array(images)
# Normalize the images to [-1, 1]
images = (images - 127.5) / 127.5
random.shuffle(images)
#images=images[:1100]

# Batch and shuffle the data
BUFFER_SIZE = 17000
BATCH_SIZE = 100

# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# Number of epochs
epochs = 100

for epoch in range(epochs):


        # Specify the image folder path
    image_folder = Path('/content/dataset/imgsd')

    # Define constants
    TOTAL_IMAGES = 17000
    IMAGES_PER_LOOP = 1000
    BATCH_SIZE = 100




    # Process images in batches
    image_paths = list(image_folder.glob('*.*'))
    random.shuffle(image_paths)

    for i in range(0, TOTAL_IMAGES, IMAGES_PER_LOOP):
        # Shuffle the image paths

        batch_paths = image_paths[i : i + IMAGES_PER_LOOP]
        batch_images = []
        for image_path in batch_paths:
            image = Image.open(image_path)
            image = image.resize((178, 218))
            image = np.array(image)
            batch_images.append(image)
        images = np.array(batch_images)
        images = (images - 127.5) / 127.5


        # Create a TensorFlow dataset
        train_dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(TOTAL_IMAGES).batch(BATCH_SIZE)


        for real_images in train_dataset:

            # Reshape real images
            real_images = tf.reshape(real_images, (100, 218*178*3))


                # Generate random noise as an input to initialize the generator
            noise = tf.random.normal([100, 20])

            # Train the discriminator first
            with tf.GradientTape() as disc_tape:
                # Generate fake images from the noise
                generated_images = model(noise, training=True)

                # Pass real images through the discriminator
                real_output = model2(real_images, training=True)

                # Pass fake images through the discriminator
                fake_output = model2(generated_images, training=True)
                batch_size = tf.shape(real_output)[0]
                num_classes = tf.shape(real_output)[1]

                # Create labels for the discriminator
                real_labels = tf.ones_like(real_output) * tf.random.uniform(shape=(batch_size, num_classes),minval=0.7, maxval=1.2)
                fake_labels = tf.zeros_like(fake_output) * tf.random.uniform(shape=(batch_size, num_classes),minval=0.0, maxval=0.3)

                # Calculate the loss for the discriminator
                real_loss = cross_entropy(real_labels, real_output)
                fake_loss = cross_entropy(fake_labels, fake_output)
                disc_loss = real_loss + fake_loss

                # Calculate the gradients for the discriminator
            gradients_of_discriminator = disc_tape.gradient(disc_loss, model2.trainable_variables)

                # Apply the gradients to the discriminator
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, model2.trainable_variables))

                # Now, train the generator
            with tf.GradientTape() as gen_tape:
                # Generate fake images from the noise
                generated_images = model(noise, training=True)

                # Pass the generated images through the discriminator
                fake_output = model2(generated_images, training=False)

                # Calculate the loss for the generator (the labels are ones here because the generator tries to fool the discriminator)
                gen_loss = generator_loss(fake_output)

            # Calculate the gradients for the generator
            gradients_of_generator = gen_tape.gradient(gen_loss, model.trainable_variables)

            # Apply the gradients to the generator
            generator_optimizer.apply_gradients(zip(gradients_of_generator, model.trainable_variables))

        # Print the loss for each epoch
            print(f'Epoch: {epoch}, Gen Loss: {gen_loss}, Disc Loss: {disc_loss}')


      # Save the generator and discriminator models after every fifth epoch
    if (epoch + 1) % 10 == 0:

        # Generate random noise vector
        noise = np.random.rand(1, 20)

        # Generate image using the existing generator model
        generated_image = model.predict(noise)

        # Rescale pixel values from [-1, 1] to [0, 1]
        generated_image = 0.5 * (generated_image + 1)

        # Reshape the generated image to (218, 178, 3)
        generated_image = generated_image.reshape((218, 178, 3))

        # Display the generated image
        plt.imshow(generated_image)
        plt.axis('off')
        plt.show()

        model.save('generator_model_{}.h5'.format(epoch + 1))

Epoch: 0, Gen Loss: 47.80029296875, Disc Loss: 1.3527719974517822
Epoch: 0, Gen Loss: 592.9833984375, Disc Loss: 122.67679595947266
Epoch: 0, Gen Loss: 0.0, Disc Loss: 118.70840454101562
Epoch: 0, Gen Loss: 455.47235107421875, Disc Loss: 741.3045654296875
Epoch: 0, Gen Loss: 547.5902099609375, Disc Loss: 37.009498596191406
Epoch: 0, Gen Loss: 8.867726022080645e-32, Disc Loss: 46.448055267333984
Epoch: 0, Gen Loss: 801.75830078125, Disc Loss: 84.33442687988281
Epoch: 0, Gen Loss: 1503.33935546875, Disc Loss: 64.425048828125
Epoch: 0, Gen Loss: 1764.20458984375, Disc Loss: 123.06332397460938
Epoch: 0, Gen Loss: 1613.2412109375, Disc Loss: 106.29163360595703
Epoch: 0, Gen Loss: 1080.776123046875, Disc Loss: 91.99787139892578
Epoch: 0, Gen Loss: 476.6784362792969, Disc Loss: 34.08921813964844
Epoch: 0, Gen Loss: 1.2538351314915182e-15, Disc Loss: 62.40019226074219
Epoch: 0, Gen Loss: 1302.627197265625, Disc Loss: 128.51234436035156
Epoch: 0, Gen Loss: 2570.51318359375, Disc Loss: 67.738883

In [1]:
import zipfile
from PIL import Image
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Replace 'path_to_zip_file' with the path of your zip file
with zipfile.ZipFile('/content/img.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

In [ ]:
# Generate random noise vector
noise = np.random.rand(1, 20)

# Generate image using the existing generator model
generated_image = model.predict(noise)

# Rescale pixel values from [-1, 1] to [0, 1]
generated_image = 0.5 * (generated_image + 1)

# Reshape the generated image to (218, 178, 3)
generated_image = generated_image.reshape((218, 178, 3))

# Display the generated image
plt.imshow(generated_image)
plt.axis('off')
plt.show()